# Dataset utils for Knowledge Graph Construction

In [ ]:
#| default_exp ml.kg.dataset

In [ ]:
#| hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#|export

def _batch_transform_webnlg(examples):
    for lex, mts in zip(examples['lex'], examples['modified_triple_sets']):
        for text in lex['text']:
            yield dict(text=text, triplets=mts['mtriple_set'][0])

def batch_transform_webnlg(examples):
    records = list(_batch_transform_webnlg(examples))
    return {
        'text': [record['text'] for record in records],
        'triplets': [record['triplets'] for record in records],
    }

In [ ]:
#|hide
from datasets import load_dataset
ds = load_dataset("web_nlg", "release_v3.0_en", split="train[:10]")
erx_ds = ds.map(batch_transform_webnlg, batched=True, remove_columns=ds.column_names)

assert 'text' in erx_ds.features
assert 'triplets' in erx_ds.features
assert isinstance(erx_ds[0]['triplets'], list)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()